# Compute dense optical flow using farneback

In [ ]:
#from python_scripts.MPI_Data_Processing.mot import *
from mot import *
from os.path import basename
import os

vid = "./RPS_05_PS_C2.wmv"
outfile = "./motion_tensors/motion_vectors_farneback3.hdf5"

In [ ]:
#Input scaled to one quater of the original size of the frames. 

In [ ]:
motob = ComputeMotion(vid,outfile,algorithm='farneback', scaled_size = (480,270), output_scaling = .25, use_multiprocessing = False)
motob.to_file()

# Compute dense optical flow using Brocks

In [1]:
#from python_scripts.MPI_Data_Processing.mot import *
from mot import *
from os.path import basename
import os

vid = "./RPS_05_PS_C2.wmv"
outfile = "./motion_vectors_brocks2.hdf5"

In [ ]:
motob = ComputeMotion(vid,outfile,algorithm='brocks', scaled_size = (480,270), output_scaling = .25)
motob.to_file()

# Extract dense optical flow from H264

In [3]:
outfile = os.getcwd()+"/motion_tensors/motion_vectors_h264.h5"
vid = os.getcwd()+"/RPS_05_PS_C2.wmv"

In [4]:
decode_motion = "/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/dense_flow_decoder/decode_motion"

In [24]:
! {decode_motion} {vid} {outfile}

Input #0, asf, from '/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/video_processing_pipeline/correlation_between_detected_motion/RPS_05_PS_C2.wmv':
  Metadata:
    WMFSDKNeeded    : 0.0.0.0000
    WMFSDKVersion   : 12.0.15063.608
    IsVBR           : 0
  Duration: 00:20:00.81, bitrate: 1609 kb/s
    Stream #0:0(eng): Video: h264 (Main) (H264 / 0x34363248), yuvj420p(pc, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 50000 kb/s, 25 fps, 25 tbr, 1k tbn, 2k tbc

**************************************************************************************
*       Tool : Dense Flow Decoder                                                    *
*     Author : Michael Gerstenberger (see copyright for version history and authors) *
*  Used Libs : FFmpeg, HDF5                                                          *
*Description : A tool to extract motion vectors from H264 videos and save them as    *
             : a tensor of size n_frames x width x height x 2 (for dx and 

## Median filter h264

In [5]:
from scipy.ndimage import median_filter

h264_median = os.getcwd() + "/motion_tensors/motion_vectors_h264_median.hdf"

f2 = tables.open_file(outfile, mode='r')
b = f2.root.motion_tensor

c = np.ndarray(b.shape)

print("This takes a while")
c[:,:,:,0] = median_filter(b[:,:,:,0], size = 2)
print("Half way done")
c[:,:,:,1] = median_filter(b[:,:,:,1], size = 2)

In [ ]:
f = tables.open_file(h264_median, 'w')
atom = tables.Atom.from_dtype(np.int32)
ds = f.create_carray(f.root, 'motion_tensor', atom, c.shape)
ds[:] = c
f.close()